In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pandas as pd
import numpy as np
import pickle

In [2]:
## load the trained model and all pickle files

## load the trained model
model=load_model('model.h5')

## load the label_encoder
with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gen= pickle.load(file)

## load the one hot_encoder
with open('one_hot_encoder_geo.pkl','rb') as file:
   one_hot_geo= pickle.load(file)

## load the scaler for standarization
with open('scaler.pkl','rb') as file:
    scaler= pickle.load(file)


In [4]:
# Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [5]:
[input_data]

[{'CreditScore': 600,
  'Geography': 'France',
  'Gender': 'Male',
  'Age': 40,
  'Tenure': 3,
  'Balance': 60000,
  'NumOfProducts': 2,
  'HasCrCard': 1,
  'IsActiveMember': 1,
  'EstimatedSalary': 50000}]

In [6]:
## tranform the data for prediction
input_df=pd.DataFrame([input_data])

In [7]:
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [17]:
## one hot encoding on geograpght column
geo_transf=one_hot_geo.transform(input_df[['Geography']]).toarray().astype(int)

In [18]:
geo_transf

array([[1, 0, 0]])

In [19]:
data=pd.DataFrame(geo_transf,columns=['France','Germany','Spain'])
input_data=pd.concat([input_df.drop('Geography',axis=1),data],axis=1)


In [20]:
input_data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,France,Germany,Spain
0,600,Male,40,3,60000,2,1,1,50000,1,0,0


In [22]:
## perform label encoding
input_data['Gender']=label_encoder_gen.transform(input_data['Gender'])

In [23]:
input_data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,France,Germany,Spain
0,600,1,40,3,60000,2,1,1,50000,1,0,0


In [24]:
scaled_data=scaler.transform(input_data)


In [25]:
scaled_data

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [35]:
prediction=model.predict(scaled_data)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step


In [37]:
prediction_proba=prediction[0][0]
if prediction_proba > 0.5:
    print('The customer is likely to churn.')
else:
    print('The customer is not likely to churn.')


The customer is not likely to churn.
